# Timestep Effect

How does the amount of past data available impact generalization performance of these models?

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
torch.multiprocessing.set_sharing_strategy('file_system')
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from pickle import load
import h5py

import DataUtils
import Masking

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Defining an RUL Transformer Model

In [5]:
class RULTransformer(nn.Module):
    def __init__(self, timesteps):
        super().__init__()
        
        self.prediction_window = 1 # only estimates the next point
        self.input_len = timesteps # timesteps (half past, half future)
        self.n_cols = 35 # 35 columns reduced
        self.embed_dim = 64 # project to a 64-dimensional space
        
        self.input_projection = nn.Linear(self.input_len*self.n_cols, self.embed_dim) 
        self.positional_embed = nn.Parameter(torch.randn(self.embed_dim)) # learned positional embeddings
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.embed_dim, nhead=4, activation="gelu", dim_feedforward=64)
        self.transformer_blocks = nn.TransformerEncoder(self.encoder_layer, num_layers=1)
        
        self.rul_head = nn.Sequential(nn.Linear(self.embed_dim, self.prediction_window)) # linear rul prediction head 
    
    def forward(self, x):
        x = x.flatten(1)
        z = self.input_projection(x)
        z = z + self.positional_embed
        z = self.transformer_blocks(z)
        z = self.rul_head(z)
        
        return z.squeeze(1)

## Timestep Sweep

In [6]:
timestep_grid = [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]

In [9]:
def train_model(timestep, trainloader):
    columns_kept = [True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               False, False, False, False, False, False, False, False, False, False,
               True, True, True, False]

    objective = nn.MSELoss()

    rul_tran = RULTransformer(timestep).to(device)

    lr = 1e-4
    n_epochs = 5
    optim = torch.optim.Adam(rul_tran.parameters(), lr=lr)
    losses = []

    for n in range(n_epochs):
        counter = 0
        for i, (X, y) in enumerate(tqdm(trainloader)):
            X = X.to(device)[:,:,columns_kept]
            y = y.to(device)
            optim.zero_grad()
            yhat = rul_tran(X.float())
            loss = objective(yhat, y.float().squeeze(1))
            loss.backward()
            losses.append(loss.cpu().detach().numpy())
            optim.step()
            counter += 1

        print("Epoch:", n+1, "Loss:",np.mean(losses[-counter:][0]))
        
    return rul_tran

def validate_model(model, testloader):
    columns_kept = [True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               False, False, False, False, False, False, False, False, False, False,
               True, True, True, False]
    objective = nn.MSELoss()
    test_mses = []
    yhats = []
    ys = []
    with torch.no_grad():
        for i, (X, y) in enumerate(tqdm(testloader)):
            yhat = model(X.float().to(device)[:,:,columns_kept])
            yhats.append(yhat.cpu())
            ys.append(y.cpu().unsqueeze(1))
            test_mse = objective(yhat, y.float().to(device))
            test_mses.append(test_mse.item())
    print("Test MSE:", np.mean(test_mses))
    print("Test RMSE:", np.sqrt(np.mean(test_mses)))
    
    return np.sqrt(np.mean(test_mses))

In [10]:
rmses = {}
for t in timestep_grid:
    batch_size = 64
    trainloader, testloader = DataUtils.get_ncmapss_dataloaders(1, n_timesteps=t, batch=batch_size, workers=8, subsampled=True)
    
    rmse_list = []
    for i in range(3):
        rul_model = train_model(t, trainloader)
        rmse = validate_model(rul_model, testloader)
        rmse_list.append(rmse)
    
    rmses[str(t)] = np.mean(rmse_list)
    print(rmses)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [00:54<00:00, 139.52it/s]


Epoch: 1 Loss: 2796.4062


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [00:58<00:00, 131.46it/s]


Epoch: 2 Loss: 35.338837


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [00:58<00:00, 130.75it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:06<00:00, 115.23it/s]


Epoch: 3 Loss: 1.0084128


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:06<00:00, 114.67it/s]


Epoch: 4 Loss: 0.44648466


 43%|█████████████████████████████████████████████████████████████████████████▍                                                                                                | 3310/7667 [00:29<00:38, 113.55it/s]

In [19]:
print(rmses)

In [21]:
x = [float(key) for key in rmses.keys()]
y = list(rmses.values())
plt.plot(x, y, marker='o')
plt.xlabel('Past Data Size')
plt.ylabel('RMS Error')
plt.title('Timestep Effect on RMSE (N-CMAPSS DS 1)')
plt.savefig('../results/past_effect_lineplot.png')
plt.show()